## Evaluation of Generators for future Energy Scenarios


In [5]:
"""
Setup environment and load the base PyPSA-Earth network for a specified country.
"""
import os
import sys
import warnings
import pypsa
import matplotlib.pyplot as plt
import geopandas as gpd
import fiona
import warnings
from pathlib import Path
import requests
import shutil
import pypsa
import copy

import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_hex
# Import all dirs
parent_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(parent_dir))
from src.paths import all_dirs
dirs = all_dirs()

# Suppress warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

# ---------------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------------
# Define country parameters
country_code = "EC"        # ISO 2-letter code (e.g., 'GH' for Ghana, 'CO' for Colombia)
country_name = "Ecuador"   # Country name
country_gadm = "ECU"       # ISO 3-letter GADM code



#Processing and Plotting all the Generation Points

Data was manually prepared from
 
https://www.ambienteyenergia.gob.ec/wp-content/uploads/2020/01/5.-PLAN-DE-EXPANSION-DE-LA-TRANSMISION.pdf

The Generation needs to be listed and mapped to the different buses of the network to be ready for coupling. 

In [6]:
import os
import pandas as pd
import unicodedata

# === Step 0: Assert the file exists ===
generation_path = dirs["data/raw/generation"]
generation_file = "generation_filtered.xlsx"
full_generation_path = os.path.join(generation_path, generation_file)
assert os.path.isfile(full_generation_path), f"❌ Generation file not found: {full_generation_path}"

# === Step 1: Load the Excel file ===
input_file = full_generation_path
df = pd.read_excel(input_file)

# === Step 2: Define the new structure ===
columns = [
    "Name", "Fueltype", "Technology", "Set", "Country", "Capacity",
    "Efficiency", "Duration", "Volume_Mm3", "DamHeight_m",
    "StorageCapacity_MWh", "DateIn", "DateRetrofit", "DateOut",
    "lat", "lon", "EIC", "projectID"
]
new_df = pd.DataFrame(columns=columns)

# === Step 3: Map available data ===
new_df["Name"] = df["Central"]
new_df["Fueltype"] = df["Tipo de Central"]
new_df["Technology"] = df["Subtipo de Central"]
new_df["Set"] = df["Sistema"]
new_df["Country"] = "Ecuador"
new_df["Capacity"] = df["Potencia Efectiva (MW)"]
new_df["lat"] = df["Latitud"]
new_df["lon"] = df["Longitud"]

# === Step 4: Clean strings ===
def clean_text(value):
    """Remove accents, replace spaces with underscores, and handle NaN safely."""
    if pd.isna(value):
        return ""
    value = str(value)
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('utf-8')
    value = value.replace(" ", "_")
    return value.strip()

for col in new_df.select_dtypes(include=['object']).columns:
    new_df[col] = new_df[col].apply(clean_text)

# === Step 5: Ensure unique names ===
def make_unique(series):
    """Append _2, _3, etc. to duplicated names."""
    counts = {}
    unique_names = []
    for name in series:
        if name not in counts:
            counts[name] = 1
            unique_names.append(name)
        else:
            counts[name] += 1
            unique_names.append(f"{name}_{counts[name]}")
    return unique_names

if new_df["Name"].duplicated().any():
    print("⚠️ Duplicate names detected — renaming...")
    new_df["Name"] = make_unique(new_df["Name"])

assert new_df["Name"].is_unique, "❌ Duplicate names still exist after renaming."

# Temp Date In so they are by default included in the evaluation

new_df["DateIn"]= 2000  
# === Step 6: Export cleaned file ===
ppl_path = dirs["data/processed/generation"]
os.makedirs(ppl_path, exist_ok=True)
ppl_file = os.path.join(ppl_path, "powerplants_existing.csv")
new_df.to_csv(ppl_file, index=False, encoding="utf-8")

ppl_existent = copy.deepcopy(new_df)

print(f"✅ New CSV file created and cleaned: {ppl_file}")
print(f"✅ All {len(new_df)} plant names are unique.")


⚠️ Duplicate names detected — renaming...
✅ New CSV file created and cleaned: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\generation\powerplants_existing.csv
✅ All 317 plant names are unique.


# Newer power plants
New power plants that according the master plan shall come into operation



In [7]:
import os
import pandas as pd
import unicodedata
import numpy as np

# === Paths ===
generation_path = dirs["data/raw/generation"]
input_file = os.path.join(generation_path, "generation_future.xlsx")
assert os.path.isfile(input_file), f"❌ Future generation file not found: {input_file}"


# === Load ===
df = pd.read_excel(input_file, sheet_name="Future")

# === Target structure ===
columns = [
    "Name",
    "Fueltype",
    "Technology",
    "Set",
    "Country",
    "Capacity",
    "Efficiency",
    "Duration",
    "Volume_Mm3",
    "DamHeight_m",
    "StorageCapacity_MWh",
    "DateIn",
    "DateRetrofit",
    "DateOut",
    "lat",
    "lon",
    "EIC",
    "projectID",
]
new_df = pd.DataFrame(columns=columns)

# === Map available data ===
new_df["Name"] = df["Proyecto / Central"]
new_df["Fueltype"] = df["Tipo"]  # keep as provided (normalized below)
new_df["Technology"] = ""  # not provided in this sheet
new_df["Set"] = "S.N.I."  # tag to distinguish dataset
new_df["Country"] = "Ecuador"
new_df["Capacity"] = pd.to_numeric(df["Potencia [MW]"], errors="coerce")
new_df["DateIn"] = pd.to_numeric(df["Año de entrada en operación"], errors="coerce")
new_df["lat"] = pd.to_numeric(df["Latitud"], errors="coerce")
new_df["lon"] = pd.to_numeric(df["Longitud"], errors="coerce")

# Empty optional fields
for c in [
    "Efficiency",
    "Duration",
    "Volume_Mm3",
    "DamHeight_m",
    "StorageCapacity_MWh",
    "DateRetrofit",
    "DateOut",
    "EIC",
    "projectID",
]:
    new_df[c] = np.nan


# === String normalization (remove accents, spaces->underscores) ===
def clean_text(value):
    if pd.isna(value):
        return ""
    value = str(value)
    value = (
        unicodedata.normalize("NFKD", value).encode("ascii", "ignore").decode("utf-8")
    )
    value = value.replace(" ", "_")
    return value.strip()


for col in ["Name", "Fueltype", "Technology", "Set", "Country", "EIC", "projectID"]:
    new_df[col] = new_df[col].apply(clean_text)


# === Ensure unique plant names ===
def make_unique(series):
    counts = {}
    out = []
    for name in series:
        if name not in counts:
            counts[name] = 1
            out.append(name)
        else:
            counts[name] += 1
            out.append(f"{name}_{counts[name]}")
    return out


if new_df["Name"].duplicated().any():
    print("⚠️ Duplicate names detected — renaming...")
    # new_df["Name"] = make_unique(new_df["Name"])

assert new_df["Name"].is_unique, "❌ Duplicate names still exist after renaming."

tech = {
    "Hidroelectrico": "Embalse",
    "Termoelectrico": "MCI",
    "Eolico": "Eolica",
    "ERNC": "Fotovoltaica",
    "Nuclear": "Nuclear",
}

for i, row in new_df.iterrows():
    new_df.at[i,"Technology"] = tech[row["Fueltype"]]

# === Export ===

# === Step 6: Export cleaned file ===
ppl_path = dirs["data/processed/generation"]
os.makedirs(ppl_path, exist_ok=True)
out_file = os.path.join(ppl_path, "powerplants_future.csv")

# Use UTF-8 (PyPSA-friendly). If you want Excel-safe accents, use encoding='utf-8-sig'.
new_df.to_csv(out_file, index=False, encoding="utf-8")
ppl_future = copy.deepcopy(new_df)
print(f"✅ Future CSV created: {out_file}")
print(f"✅ Rows: {len(new_df)} | Unique names: {new_df['Name'].is_unique}")

✅ Future CSV created: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\generation\powerplants_future.csv
✅ Rows: 30 | Unique names: True


Combine all power plants into a single File


In [8]:
import os
import pandas as pd

# === Step 1: Define the output path ===
ppl_path = dirs["data/processed/generation"]
os.makedirs(ppl_path, exist_ok=True)
out_file = os.path.join(ppl_path, "powerplants_all.csv")
all_ppl_file = out_file
# === Step 2: Concatenate vertically ===
# (both DataFrames must have the same columns)
combined_df = pd.concat([ppl_existent, ppl_future], axis=0, ignore_index=True)

# === Step 3: Verify structure ===
print(f"🧩 Combined shape: {combined_df.shape}")
print(f"📋 Columns: {list(combined_df.columns)}")

# === Step 4: Export to CSV ===
# Use utf-8-sig for Excel compatibility if needed
combined_df.to_csv(out_file, index=False, encoding="utf-8-sig")

print(f"✅ Combined power plants file saved at:\n   {out_file}")
print(f"✅ Total plants: {len(combined_df)}")


🧩 Combined shape: (347, 18)
📋 Columns: ['Name', 'Fueltype', 'Technology', 'Set', 'Country', 'Capacity', 'Efficiency', 'Duration', 'Volume_Mm3', 'DamHeight_m', 'StorageCapacity_MWh', 'DateIn', 'DateRetrofit', 'DateOut', 'lat', 'lon', 'EIC', 'projectID']
✅ Combined power plants file saved at:
   c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\generation\powerplants_all.csv
✅ Total plants: 347
